In [ ]:
import pandas
import numpy as np
test=pandas.read_csv("../data/df_test.csv")
train=pandas.read_csv("../data/df_train.csv")

Counter({2: 53, 1: 49, 7: 20, 3: 12, 5: 9, 6: 6})


In [ ]:
def matrix_data(tt): 
    tt_nl=tt.loc[:, tt.columns != "type"]
    tt_l=tt.loc[:, tt.columns == "type"]
    return(np.array(tt_nl),tt_l)

In [ ]:
train.shape

(149, 10)

In [ ]:
def activation(matrix):
    #apply activation here 
    return matrix

In [ ]:
from collections import Counter

In [ ]:
print(Counter(test["type"]))

Counter({2: 23, 1: 21, 7: 9, 3: 5, 5: 4, 6: 3})


In [ ]:
##normalise data before 

In [ ]:
def next_layer(previous_layer,relevant_weights):
    bias=0.5
    train_nl,train_l=matrix_data(train)
    weights=np.random.rand(train_nl.shape[1],10)
    activation(train_nl@weights+bias)

In [ ]:
weights = np.random.rand(9, 149)
train @ weights

ValueError: Dot product shape mismatch, (149, 10) vs (9, 149)

In [ ]:
def create weights 

In [ ]:
class network:
    def __init__(self, layers_info):
        #self.n_features = n_features
        ##self.bias = np.random.rand(neurons,1)
        self.layers_info = layers_info
        self.Network=[]
    
    def forward_pass(self,input_data):
        #create new layers here 
        for l_info in self.layers_info:
            input_data = np.array(input_data)
            new_layer = layer(l_info, input_data.shape[1])
            new_layer.calculation(input_data)
            self.Network.append(new_layer)
            input_data = new_layer.next_layer_input
        output_layer= layer(6, input_data.shape[1])
        output_layer.calculation(input_data,"softmax")
        self.Network.append(output_layer)
    
    def mse_loss(y_true, y_pred):
        return ((y_true-y_pred)**2).mean()
        
    def get_layer(self, i):
        return self.network[i]
    

class layer:
    def __init__(self,number_of_neurons, n_prev_neurons):
        self.n = number_of_neurons
        self.weights = np.random.rand(n_prev_neurons, self.n)#noahs suggestion[[Var(random.random()) for _ i range(number_of_neurons)] for _ in range(n_pr)]
        self.bias = 0.5
        self.next_layer_input = None
    
    def activation(self,data,activate):
        if activate == "ReLU":
            return np.maximum(data,0.0)
        elif activate == "sigmoid":
            return  1.0 / (1.0 + np.exp(-data))
        elif activate == "softmax":
            data=np.array(data,dtype=np.float128)
            return np.exp(data)/(np.exp(data).sum(axis=1).reshape(-1,1))
        else:
            print("invalid activaiton function name")
        #more here if needed
        #choose activation function here 
        
    
    def calculation(self, data, activate = "sigmoid"):
        self.activate=activate
        self.next_layer_input = self.activation(data @ self.weights + bias, activate)

In [ ]:
from sklearn.preprocessing import StandardScaler
norm = StandardScaler()
x = norm.fit_transform(matrix_data(train)[0])
testingthing=network([10,3,4,7])
testingthing.forward_pass(x)


In [ ]:

output=testingthing.Network[4].next_layer_input
np.argwhere(output==np.amax(output,axis=1).reshape(output.shape[0],1))[:,1]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
np.ravel(matrix_data(train)[1])

array([2, 2, 2, 2, 1, 2, 2, 5, 2, 7, 1, 2, 7, 1, 1, 1, 7, 2, 2, 1, 2, 7,
       7, 6, 3, 2, 6, 2, 6, 7, 1, 2, 6, 2, 7, 1, 2, 1, 1, 2, 7, 2, 1, 1,
       2, 1, 3, 2, 2, 2, 2, 1, 7, 7, 1, 5, 1, 1, 1, 1, 1, 3, 1, 2, 3, 7,
       2, 2, 2, 1, 3, 6, 5, 2, 1, 6, 2, 2, 7, 2, 1, 1, 3, 5, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 3, 1, 3, 7, 2, 5, 1,
       2, 1, 1, 1, 7, 3, 1, 3, 5, 2, 2, 5, 7, 2, 1, 7, 2, 3, 7, 7, 2, 1,
       7, 1, 1, 2, 2, 2, 5, 2, 5, 1, 2, 1, 1, 2, 7, 1, 1])

In [ ]:
np.amax(output,axis=1)

array([0.35629773, 0.35627203, 0.35609579, 0.35611337, 0.35607858,
       0.35607057, 0.35623628, 0.35614003, 0.35602834, 0.35630093,
       0.35622909, 0.35619161, 0.35633964, 0.35596704, 0.35608197,
       0.35607274, 0.3563268 , 0.35606341, 0.35600757, 0.35610186,
       0.3560863 , 0.35622709, 0.35617913, 0.35606704, 0.3560053 ,
       0.35619109, 0.35600853, 0.35608212, 0.35611777, 0.35619342,
       0.35626186, 0.35602851, 0.35602522, 0.3560683 , 0.35627142,
       0.3560628 , 0.35607133, 0.35621684, 0.3560652 , 0.35621919,
       0.3561796 , 0.35607542, 0.35607465, 0.35603079, 0.3562459 ,
       0.3562639 , 0.35599144, 0.35629977, 0.35598286, 0.35599383,
       0.35609562, 0.35625156, 0.35622902, 0.3562803 , 0.35618193,
       0.35615526, 0.35608186, 0.35620765, 0.35617608, 0.35607873,
       0.35618327, 0.3563083 , 0.3560579 , 0.35627167, 0.35599832,
       0.35621373, 0.35611709, 0.35612123, 0.35623599, 0.35622148,
       0.35601378, 0.35605507, 0.35629814, 0.35611959, 0.35606

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=58866f15-1cff-4fef-9525-5c3070562370' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>